---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 1 - Creating and Manipulating Graphs

Eight employees at a small company were asked to choose 3 movies that they would most enjoy watching for the upcoming company movie night. These choices are stored in the file `Employee_Movie_Choices.txt`.

A second file, `Employee_Relationships.txt`, has data on the relationships between different coworkers. 

The relationship score has value of `-100` (Enemies) to `+100` (Best Friends). A value of zero means the two employees haven't interacted or are indifferent.

Both files are tab delimited.

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
from networkx.algorithms import bipartite


# This is the set of employees
employees = set(['Pablo',
                 'Lee',
                 'Georgia',
                 'Vincent',
                 'Andy',
                 'Frida',
                 'Joan',
                 'Claude'])

# This is the set of movies
movies = set(['The Shawshank Redemption',
              'Forrest Gump',
              'The Matrix',
              'Anaconda',
              'The Social Network',
              'The Godfather',
              'Monty Python and the Holy Grail',
              'Snakes on a Plane',
              'Kung Fu Panda',
              'The Dark Knight',
              'Mean Girls'])


# you can use the following function to plot graphs
# make sure to comment it out before submitting to the autograder
# def plot_graph(G, weight_name=None):
#     '''
#     G: a networkx G
#     weight_name: name of the attribute for plotting edge weights (if G is weighted)
#     '''
#     %matplotlib notebook
#     import matplotlib.pyplot as plt
    
#     plt.figure()
#     pos = nx.spring_layout(G)
#     edges = G.edges()
#     weights = None
    
#     if weight_name:
#         weights = [int(G[u][v][weight_name]) for u,v in edges]
#         labels = nx.get_edge_attributes(G,weight_name)
#         nx.draw_networkx_edge_labels(G,pos,edge_labels=labels)
#         nx.draw_networkx(G, pos, edges=edges, width=weights);
#     else:
#         nx.draw_networkx(G, pos, edges=edges);

### Question 1

Using NetworkX, load in the bipartite graph from `Employee_Movie_Choices.txt` and return that graph.

*This function should return a networkx graph with 19 nodes and 24 edges*

In [2]:
def answer_one():
    B = nx.Graph()
    B.add_nodes_from(employees, bipartite=0,type='employee') # Add the node attribute “bipartite”
    B.add_nodes_from(movies, bipartite=1,type='movies')
#     B.add_edges_from(edges)
#     G4 = nx.read_edgelist('Employee_Movie_Choices.txt',delimiter='\t')
    G_df = pd.read_csv('Employee_Movie_Choices.txt', sep='\t',
                   header=None, names=['employees', 'movies'])
    G_df=G_df.drop(G_df.index[0])
    edges=G_df.values.tolist()
#     G5 = nx.from_pandas_dataframe(G_df, 'employees', 'movies')
#     G5.edges(data=True)
    B.add_edges_from(edges)
    return B
# print(answer_one())

### Question 2

Using the graph from the previous question, add nodes attributes named `'type'` where movies have the value `'movie'` and employees have the value `'employee'` and return that graph.

*This function should return a networkx graph with node attributes `{'type': 'movie'}` or `{'type': 'employee'}`*

In [3]:
def answer_two():
    G=answer_one()
#     G= nx.read_edgelist('Employee_Movie_Choices.txt',delimiter='\t')
# #     attrs = {'type': 'movie', 'type': 'employee'}
    nx.set_node_attributes(G, type,['movie','employee'])
    return answer_one()
# print(answer_two())

### Question 3

Find a weighted projection of the graph from `answer_two` which tells us how many movies different pairs of employees have in common.

*This function should return a weighted projected graph.*

In [4]:
def answer_three():
    G = bipartite.weighted_projected_graph(answer_two(), employees,ratio=True)
    # Your Code Here
    
    return G
# print(answer_three().nodes(data=True))

### Question 4

Suppose you'd like to find out if people that have a high relationship score also like the same types of movies.

Find the Pearson correlation ( using `DataFrame.corr()` ) between employee relationship scores and the number of movies they have in common. If two employees have no movies in common it should be treated as a 0, not a missing value, and should be included in the correlation calculation.

*This function should return a float.*

In [12]:
def aux_four():
    G = bipartite.weighted_projected_graph(answer_two(), employees,ratio=True)
    df = pd.DataFrame(G.edges(data=True), columns=['person1', 'person2', 'weight'])
    df['weight'] = df['weight'].map(lambda x: x['weight'])
    # Your Code Here
    G_df = pd.read_csv('Employee_Relationships.txt', sep='\t',
                   header=None, names=['person1', 'person2','weight'])
    G_df=G_df.drop(G_df.index[0])
    df_mascara=df[['person1','person2','weight']]
    G_df_mascara=G_df[['person1','person2','weight']]

    G_df_mascara=G_df_mascara.reset_index(drop=True)
    df_mascara=df_mascara.reset_index(drop=True)
    df_mascara=df_mascara.sort_index().sort_index(axis=1)
    G_df_mascara=G_df_mascara.sort_index(axis=1)
    return df_mascara, G_df_mascara

def prepara_reflexivo_filmes_comum():
    reflexivo= pd.concat([pd.DataFrame(aux_four()[0]['person2']),pd.DataFrame(aux_four()[0]['person1'])],axis=1)
    reflexivo['weight']=aux_four()[0]['weight']
    reflexivo.columns=['person1','person2','weight']
    return pd.concat([aux_four()[0],reflexivo],names=['person1','person2','weight'],axis=0,ignore_index=True)


def answer_four_aux():
        colegas_c_filmes_em_comum= prepara_reflexivo_filmes_comum()
        relacoes=aux_four()[1]
#         df = relacoes.loc[colegas_c_filmes_em_comum['person1'] != relacoes['person1']].values
        return relacoes,colegas_c_filmes_em_comum
#         return
    
def answer_four():
#     return 0.78839622217334737
#     return 0.8014749455697413
    list_relacoes=answer_four_aux()[0][['person1','person2']].values.tolist()
    list_filmes_em_comum=answer_four_aux()[1][['person1','person2']].values.tolist()
    nao_tem_filmes=list()
    [nao_tem_filmes.append(y) for y in list_relacoes if y not in list_filmes_em_comum]
    df_nao=pd.DataFrame(nao_tem_filmes)
    df_nao.columns=['person1','person2']
    df_nao['weight']=0
    df_nao_sim= pd.concat([df_nao,answer_four_aux()[1]],axis=0,ignore_index=True)
    lista_sim_nao=df_nao_sim[['person1','person2','weight']].values.tolist()
    relacoes_e_filmes=pd.DataFrame([y for y in lista_sim_nao if y[0:2] in list_relacoes])
    df2=pd.DataFrame(relacoes_e_filmes)
    df2.columns=['person1','person2','weight2']
    df3=answer_four_aux()[0]
    df_merge_col = pd.merge(df2, df3, on=['person1','person2'],how='outer')
    return float(df_merge_col['weight'].corr(df_merge_col['weight2']))
#     return df_merge_col

answer_four()

0.8014749455697415